# Recursion
Recursive functions call themselves repeatedly until a base case is reached.

In [ ]:
# A recursive function could be represented abstractly as:
'''
function foo(input):
    if input == BASE_CASE:
        return DIRECT_ANSWER
    else:
        return foo(input.part1) + foo(input.part2)
'''

To deal with recursion, the computer uses instruction pointers to keep track of where it is in the code. It stores return addresses as new stack frames in the call stack memory. The call stack the 'pops' the frame from the top of the stack when it is returned to. <b>Stack overflow</b> occurs when the call stack gets too large. Stack memory is reserved in advance as a fixed amount and programs may crash if they can no longer handle function calls properly.

Python has a default limit of 1000 stack frames, which can easily be exhausted when using recursive functions.

#### Robot instructions
A robot is given a sequence of instructions, consisting of the characters 'L' (left), 'R'(right) and '2', which means <i>perform all subsequent instructions twice, but skipping the instruction that immediately follows the second time round</i> ('2' never occurs at the end of the sequence). Output a string of the 'L' and 'R' moves the robot should perform.

In [99]:
def robot_moves(seq):
    if len(seq) == 0:
        return ''
    if seq[0] == '2':
        return robot_moves(seq[1:]) + robot_moves(seq[2:])
    else:
        return seq[0] + robot_moves(seq[1:])

In [121]:
seq = 'LL'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '2LR'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '2L'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = '22LR'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")
seq = 'LL2R2L'
print(f"Given input sequence {seq}, robot will move: {robot_moves(seq)}.")

Given input sequence LL, robot will move: LL.
Given input sequence 2LR, robot will move: LRR.
Given input sequence 2L, robot will move: L.
Given input sequence 22LR, robot will move: LRRLR.
Given input sequence LL2R2L, robot will move: LLRLL.


In [115]:
# The above solution contains inefficiency insofar as it creates copies of seq at with every recursion. 
# It also produces multiple strings which are then concatenated to make new strings.
# The following implementation solves these two issues.

def robot_moves(seq):
    res = []
    move(seq, 0, res)
    return ''.join(res)

def move(seq, idx, res):
    if idx == len(seq):
        return
    if seq[idx] == '2':
        move(seq, idx+1, res)
        move(seq, idx+2, res)
    else:
        res.append(seq[idx])
        move(seq, idx+1, res)

In [119]:
# This could also be written with the helper function embedded within the main function. This makes it clearer that
# seq and res are being shared and not constantly copied.

def robot_moves(seq):
    res = []
    # helper function
    def move(idx):
        if idx == len(seq):
            return
        if seq[idx] =='2':
            move(idx+1)
            move(idx+2)
        else:
            res.append(seq[idx])
            move(idx+1)
    # end of helper function
    move(0)
    return ''.join(res)